In [2]:
import os
import pandas as pd

BASE_DIR = "output_csv"
sample_df = pd.read_csv("Sample.csv")
sample_cols = sample_df.columns.tolist()

# Remove metadata columns from column matching
metadata_cols = ["Year", "Region", "Province", "Municipality"]
data_cols = [col for col in sample_cols if col not in metadata_cols]

all_data = []

for year in os.listdir(BASE_DIR):
    year_path = os.path.join(BASE_DIR, year)
    if not os.path.isdir(year_path):
        continue

    for region in os.listdir(year_path):
        region_path = os.path.join(year_path, region)
        if not os.path.isdir(region_path):
            continue

        for province in os.listdir(region_path):
            province_path = os.path.join(region_path, province)
            if not os.path.isdir(province_path):
                continue

            for municipality in os.listdir(province_path):
                mun_path = os.path.join(province_path, municipality)
                if not os.path.isdir(mun_path):
                    continue

                csv_name = f"{municipality}.csv"
                csv_path = os.path.join(mun_path, csv_name)

                if os.path.exists(csv_path):
                    try:
                        df = pd.read_csv(csv_path)

                        # Keep only expected columns, ignore extras
                        df = df[[col for col in data_cols if col in df.columns]]

                        # Add folder-derived metadata
                        df["Year"] = year
                        df["Region"] = region
                        df["Province"] = province
                        df["Municipality"] = municipality

                        # Reorder columns to match Sample.csv
                        df = df[sample_cols]

                        all_data.append(df)
                    except Exception as e:
                        print(f"Failed to read {csv_path}: {e}")

final_df = pd.concat(all_data, ignore_index=True)
# Drop rows where Ownership is 'TOTAL' (case-sensitive match)
final_df = final_df[final_df["Ownership"] != "TOTAL"]
final_df.to_csv("Final.csv", index=False)
print("Saved Final.csv")


Saved Final.csv
